In [5]:
# TensorSpeech/TensorFlowASR
# TensorFlowASR/scripts/create_librispeech_trans.py

import os
import glob
import argparse
import librosa
from tqdm.auto import tqdm
import unicodedata

from tensorflow_asr.utils.utils import preprocess_paths

parser = argparse.ArgumentParser(prog="Setup LibriSpeech Transcripts")

parser.add_argument("--dir", "-d", type=str, default=None, help="Directory of dataset")

parser.add_argument("--output", type=str, default=None, help="The output .tsv transcript file path")

args = parser.parse_args()

assert args.dir and args.output

args.dir = preprocess_paths(args.dir)
args.output = preprocess_paths(args.output)

transcripts = []

text_files = glob.glob(os.path.join(args.dir, "**", "*.txt"), recursive=True)

for text_file in tqdm(text_files, desc="[Loading]"):
    current_dir = os.path.dirname(text_file)
    with open(text_file, "r", encoding="utf-8") as txt:
        lines = txt.read().splitlines()
    for line in lines:
        line = line.split(" ", maxsplit=1)
        audio_file = os.path.join(current_dir, line[0] + ".flac")
        y, sr = librosa.load(audio_file, sr=None)
        duration = librosa.get_duration(y, sr)
        text = unicodedata.normalize("NFC", line[1].lower())
        transcripts.append(f"{audio_file}\t{duration}\t{text}\n")

with open(args.output, "w", encoding="utf-8") as out:
    out.write("PATH\tDURATION\tTRANSCRIPT\n")
    for line in tqdm(transcripts, desc="[Writing]"):
        out.write(line)

usage: Setup LibriSpeech Transcripts [-h] [--dir DIR] [--output OUTPUT]
Setup LibriSpeech Transcripts: error: unrecognized arguments: -f C:\Users\niping1\AppData\Roaming\jupyter\runtime\kernel-ef196151-8986-4374-ac92-a029f3959657.json


SystemExit: 2

In [1]:
# TensorFlowASR/examples/conformer/train_conformer.py /


import os
import math
import argparse
import pandas as pd
from tensorflow_asr.utils import setup_environment, setup_strategy

setup_environment()
import tensorflow as tf

DEFAULT_YAML = pd.read_csv(r"C:\Users\niping1\OneDrive\config_modified.yml")
# DEFAULT_YAML = os.path.basename("C:\Users\niping1\OneDrive\config.yml")

tf.keras.backend.clear_session()

parser = argparse.ArgumentParser(prog="Conformer Training")

parser.add_argument("--config", type=str, default=DEFAULT_YAML, help="The file path of model configuration file")

parser.add_argument("--max_ckpts", type=int, default=10, help="Max number of checkpoints to keep")

parser.add_argument("--tfrecords", default=False, action="store_true", help="Whether to use tfrecords")

parser.add_argument("--tbs", type=int, default=None, help="Train batch size per replica")

parser.add_argument("--ebs", type=int, default=None, help="Evaluation batch size per replica")

parser.add_argument("--devices", type=int, nargs="*", default=[0], help="Devices' ids to apply distributed training")

parser.add_argument("--mxp", default=False, action="store_true", help="Enable mixed precision")

args = parser.parse_args()

tf.config.optimizer.set_experimental_options({"auto_mixed_precision": args.mxp})

strategy = setup_strategy(args.devices)

from tensorflow_asr.configs.config import Config
from tensorflow_asr.datasets.asr_dataset import ASRTFRecordDataset, ASRSliceDataset
from tensorflow_asr.featurizers.speech_featurizers import TFSpeechFeaturizer
from tensorflow_asr.featurizers.text_featurizers import CharFeaturizer
from tensorflow_asr.runners.transducer_runners import TransducerTrainer
from tensorflow_asr.models.conformer import Conformer
from tensorflow_asr.optimizers.schedules import TransformerSchedule

config = Config(args.config)
speech_featurizer = TFSpeechFeaturizer(config.speech_config)
text_featurizer = CharFeaturizer(config.decoder_config)

if args.tfrecords:
    train_dataset = ASRTFRecordDataset(
        speech_featurizer=speech_featurizer, text_featurizer=text_featurizer,
        **vars(config.learning_config.train_dataset_config))
    eval_dataset = ASRTFRecordDataset(
        speech_featurizer=speech_featurizer, text_featurizer=text_featurizer,
        **vars(config.learning_config.eval_dataset_config))
else:
    train_dataset = ASRSliceDataset(
        speech_featurizer=speech_featurizer, text_featurizer=text_featurizer,
        **vars(config.learning_config.train_dataset_config))
    eval_dataset = ASRSliceDataset(
        speech_featurizer=speech_featurizer, text_featurizer=text_featurizer,
        **vars(config.learning_config.eval_dataset_config))

conformer_trainer = TransducerTrainer(config=config.learning_config.running_config, text_featurizer=text_featurizer, strategy=strategy)

with conformer_trainer.strategy.scope():
    # build model
    conformer = Conformer(**config.model_config, vocabulary_size=text_featurizer.num_classes)
    conformer._build(speech_featurizer.shape)
    conformer.summary(line_length=120)

    optimizer_config = config.learning_config.optimizer_config
    optimizer = tf.keras.optimizers.Adam(
        TransformerSchedule(
            d_model=conformer.dmodel,
            warmup_steps=optimizer_config["warmup_steps"],
            max_lr=(0.05 / math.sqrt(conformer.dmodel))
        ),
        beta_1=optimizer_config["beta1"],
        beta_2=optimizer_config["beta2"],
        epsilon=optimizer_config["epsilon"])

conformer_trainer.compile(model=conformer, optimizer=optimizer,
                          max_to_keep=args.max_ckpts)

conformer_trainer.fit(train_dataset, eval_dataset, train_bs=args.tbs, eval_bs=args.ebs)

usage: Conformer Training [-h] [--config CONFIG] [--max_ckpts MAX_CKPTS]
                          [--tfrecords] [--tbs TBS] [--ebs EBS]
                          [--devices [DEVICES [DEVICES ...]]] [--mxp]
Conformer Training: error: unrecognized arguments: -f C:\Users\niping1\AppData\Roaming\jupyter\runtime\kernel-570d6ece-794c-4211-9a2c-430851990834.json


SystemExit: 2

In [9]:
# TensorFlowASR/examples/conformer/test_subword_conformer.py /

import os
import argparse
import pandas as pd
from tensorflow_asr.utils import setup_environment, setup_devices

setup_environment()
import tensorflow as tf


# DEFAULT_YAML = pd.read_csv(r"C:\Users\niping1\OneDrive\config_modified.yml")
# DEFAULT_YAML = os.path.join(os.path.abspath(os.path.dirname(__file__)), "config_modified.yml")
current_folder = globals()['_dh'][0]

# Calculating path to the input data
data_location = os.path.join(current_folder,'config_modified.yml')


tf.keras.backend.clear_session()

parser = argparse.ArgumentParser(prog="Conformer Testing")

parser.add_argument("--config", type=str, default=DEFAULT_YAML, help="The file path of model configuration file")

parser.add_argument("--saved", type=str, default=None, help="Path to saved model")

parser.add_argument("--tfrecords", default=False, action="store_true", help="Whether to use tfrecords as dataset")

parser.add_argument("--mxp", default=False, action="store_true", help="Enable mixed precision")

parser.add_argument("--sentence_piece", default=False, action="store_true", help="Whether to use `SentencePiece` model")

parser.add_argument("--device", type=int, default=0, help="Device's id to run test on")

parser.add_argument("--cpu", default=False, action="store_true", help="Whether to only use cpu")

parser.add_argument("--subwords", type=str, default=None, help="Path to file that stores generated subwords")

parser.add_argument("--output_name", type=str, default="test", help="Result filename name prefix")

args = parser.parse_args()

tf.config.optimizer.set_experimental_options({"auto_mixed_precision": args.mxp})

setup_devices([args.device], cpu=args.cpu)

from tensorflow_asr.configs.config import Config
from tensorflow_asr.datasets.asr_dataset import ASRTFRecordDataset, ASRSliceDataset
from tensorflow_asr.featurizers.speech_featurizers import TFSpeechFeaturizer
from tensorflow_asr.featurizers.text_featurizers import SubwordFeaturizer, SentencePieceFeaturizer
from tensorflow_asr.runners.base_runners import BaseTester
from tensorflow_asr.models.conformer import Conformer

config = Config(args.config)
speech_featurizer = TFSpeechFeaturizer(config.speech_config)

if args.sentence_piece:
    print("Loading SentencePiece model ...")
    text_featurizer = SentencePieceFeaturizer.load_from_file(config.decoder_config, args.subwords)
elif args.subwords and os.path.exists(args.subwords):
    print("Loading subwords ...")
    text_featurizer = SubwordFeaturizer.load_from_file(config.decoder_config, args.subwords)
else:
    raise ValueError("subwords must be set")

tf.random.set_seed(0)
assert args.saved

if args.tfrecords:
    test_dataset = ASRTFRecordDataset(
        speech_featurizer=speech_featurizer, text_featurizer=text_featurizer,
        **vars(config.learning_config.test_dataset_config)
    )
else:
    test_dataset = ASRSliceDataset(
        speech_featurizer=speech_featurizer, text_featurizer=text_featurizer,
        **vars(config.learning_config.test_dataset_config)
    )

# build model
conformer = Conformer(**config.model_config, vocabulary_size=text_featurizer.num_classes)
conformer._build(speech_featurizer.shape)
conformer.load_weights(args.saved)
conformer.summary(line_length=120)
conformer.add_featurizers(speech_featurizer, text_featurizer)

conformer_tester = BaseTester(
    config=config.learning_config.running_config,
    output_name=args.output_name
)
conformer_tester.compile(conformer)
conformer_tester.run(test_dataset)


usage: Conformer Testing [-h] [--config CONFIG] [--saved SAVED] [--tfrecords]
                         [--mxp] [--sentence_piece] [--device DEVICE] [--cpu]
                         [--subwords SUBWORDS] [--output_name OUTPUT_NAME]
Conformer Testing: error: unrecognized arguments: -f C:\Users\niping1\AppData\Roaming\jupyter\runtime\kernel-570d6ece-794c-4211-9a2c-430851990834.json


SystemExit: 2

In [4]:
# TensorFlowASR/scripts/generate_vocab_subwords.py 

import os
import argparse
import pandas as pd

from tensorflow_asr.configs.config import Config
from tensorflow_asr.featurizers.text_featurizers import SubwordFeaturizer, TFSubwordFeaturizer

DEFAULT_YAML = pd.read_csv(r"C:\Users\niping1\OneDrive\config.yml")
#DEFAULT_YAML = os.path.join(os.path.abspath(os.path.dirname(__file__)), "config.yml")

parser = argparse.ArgumentParser(prog="Vocab Training with Subwords")

parser.add_argument("corpus", nargs="*", type=str, default=[], help="Transcript files for generating subwords")

parser.add_argument("--config", type=str, default=DEFAULT_YAML, help="The file path of model configuration file")

parser.add_argument("--use_tf", default=False, action="store_true", help="Whether to use tf subwords")

parser.add_argument("--output_file", type=str, default=None, help="Path to file that stores generated subwords")

args = parser.parse_args()

config = Config(args.config)

print("Generating subwords ...")

if not args.use_tf:
    text_featurizer = SubwordFeaturizer.build_from_corpus(config.decoder_config, args.corpus)
    text_featurizer.save_to_file(args.output_file)
else:
    TFSubwordFeaturizer.build_from_corpus(config.decoder_config, args.corpus, output_file=args.output_file)

usage: Vocab Training with Subwords [-h] [--config CONFIG] [--use_tf]
                                    [--output_file OUTPUT_FILE]
                                    [corpus [corpus ...]]
Vocab Training with Subwords: error: unrecognized arguments: -f


SystemExit: 2

C:\Users\niping1\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
